## Research on the reliability of bank clients

The goal: to study whether the marital status and the number of children of the bank client affect the fact of repayment of the loan on time. Input data from the bank is statistics on the solvency of clients.


### Conclusion

- there is a direct relationship between the presence of children and the return of the loan on time. The fewer children a client has, the higher the likelihood of a timely return of loan funds
- there is a relationship between marital status and loan repayment on time. Clients from the category "not married" are the worst to repay loans, and people who are married vice versa
- the relationship between the level of income and loan repayment on time has not been identified
- the probability of debt on a real estate loan is minimal. The worst returns are loans taken for education and a car


### Key concepts

Pandas, PyMystem3, Python, lemmatization, Data Preprocessing


### Project execution plan


1. [Explore general information about data](#step1)


2. [Data preprocessing](#step2)


- define and fill in the missing values
- remove duplicates
- select the lemmas in the values of the column with the purpose of obtaining a loan
- categorize the data


3. [Answer the questions:](#step3)


- is there a relationship between the presence of children and the return of the loan on time?
- is there a relationship between marital status and loan repayment on time?
- is there a relationship between income level and loan repayment on time?
- how do different purposes of a loan affect its return on time?


4. [Write a general conclusion](#step4)

### Dataset description

**The table has the following 12 columns:**

1. children — the number of children in the family
2. days_employed — the total length of service in days
3. dob_years — a client's age in years
4. education — уровень образования клиента
5. education_id — идентификатор уровня образования
6. family_status — семейное положение
7. family_status_id — идентификатор семейного положения
8. gender — пол клиента
9. income_type — тип занятости
10. debt — имел ли задолженность по возврату кредитов
11. total_income — ежемесячный доход
12. purpose — цель получения кредита

### Let's open a file and examine the general information ¶ <a id="step1"></a>   

In [1]:
# import the libraries necessary for the work
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
data_clients = pd.read_csv('/home/ilya/Desktop/Datasets/data_bank_clients.csv')

In [3]:
data_clients.info()
display(data_clients.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


The table has 21525 rows and 12 columns. Data type in the table: float64 (2 columns), int64 (5 columns), object (5 columns). The columns 'days_employed' and 'total_income' have fewer rows than all the others (19351), which indicates gaps. The gaps could arise because the bank was waiting for information from customers, or these gaps were deliberately made due to the fact that the information is outdated.

There are negative values in the 'days_employed' column. Moreover, in the 4th line, the experience corresponds to almost 100 years. This means that the data does not correspond to reality. An error may have occurred while uploading data. We should contact the developers.

In [4]:
data_clients.describe()
# using the method describe(), let's see what deviations are in the data

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


It can be seen that the column "children" has the value "-1" and "20". In the first case, replace the value with a positive one. In the second case, it is obvious that zero is printed erroneously. Replace "20" with "2". We use the method replace().

In [5]:
data_clients ['children']=data_clients['children'].replace(-1, 1)
data_clients ['children']=data_clients['children'].replace(20, 2)
data_clients.describe()
# using the method describe (), make sure that the data has been replaced successfully

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.479721,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,0.755528,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Data preprocessing <a id="step2"></a>   
#### Define and fill in the missing values

In [7]:
print(data_clients.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


The number of skipped rows in the 'days_employed' and 'total_income' columns are the same. This indicates that the omissions were made in the same places. Let's check this assumption.

In [8]:
len(data_clients[(data_clients['days_employed'].isna()==True) & (data_clients['total_income'].isna()==True)])

2174

The missing values in the 'days_employed' column replace with zeros, because the data is initially incorrect (for this we use the method fillna ()). On this, we will leave the column with work experience. The credit department of the bank is not interested in what effect the borrower's experience has on the return of credit funds.

In [9]:
data_clients['days_employed'] = data_clients['days_employed'].fillna(value=0)
# the missing values replaced with zeros
data_clients.info()
# method info(), make sure the replacement was successful

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


From the data, can be seen that among the borrowers there are only 2 people are unemployed, 1 is on maternity leave and 1 is a student. This confirms the assumption that the omissions in the table are not due to lack of income.
The type of employment directly affects the level of income. Obviously, a student has a lower income level than an employee of the company. Let's group by the income_type column and calculate the income median for each type of employment, sort in ascending order.

In [11]:
data_median_total_income = data_clients.groupby('income_type')['total_income'].median().sort_values()
print(data_median_total_income)

income_type
в декрете           53829.130729
студент             98201.625314
пенсионер          118514.486412
безработный        131339.751676
сотрудник          142594.396847
госслужащий        150447.935283
компаньон          172357.950966
предприниматель    499163.144947
Name: total_income, dtype: float64


Instead of gaps, let us assign a median to each occupancy type.

In [12]:
# Create a function that accepts a dataset and type of employment, 
# and then replaces empty values with median values

def fill_by_median(data_clients, job_type):
    data_clients.loc[data_clients['income_type'] == job_type,'total_income'] = data_clients\
    .loc[data_clients['income_type'] == 
job_type,'total_income'].fillna(data_median_total_income[job_type])
    
# Apply the function 
fill_by_median(data_clients, 'сотрудник')

fill_by_median(data_clients, 'компаньон')

fill_by_median(data_clients, 'пенсионер')

fill_by_median(data_clients, 'госслужащий')

fill_by_median(data_clients, 'предприниматель')
data_clients.info()
# use the method info(), male sure the replacement was successful

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [13]:
# after transformations check the number of skipped rows in the 'days_employed' and 'total_income' columns
print(data_clients.isnull().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Conclusion on this point:** there were gaps in 2 columns: 'days_employed' and 'total_income'. In the first column, the missing values were replaced with zeros (in the bank's request, this column does not correlate with the fact that the loan is repaid on time). In addition, it is obvious that the data for the entire column is incorrect, as described above. For the 'total_income' column, we calculated the income median for each type of employment and replaced the missing values with it. For income, the method median () takes precedence over the method mean (). This is due to the fact that earnings can vary greatly and the average value can be used incorrectly.

#### Data type replacement

In [14]:
# replace the float data type with integers using the method astype () with an int argument
data_clients['days_employed'] = data_clients['days_employed'].astype(int)
data_clients['total_income'] = data_clients['total_income'].astype(int)
# using the method info() check the data type in the table after transformation
data_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Conclusion on this point:** there were 2 columns with the float data ('days_employed' and 'total_income'). Work experience assumes whole numbers, and decimal data in income can be neglected and the perception of numbers can be improved. To convert float data into integers, we used the method astype () and converted the experience and income into integers. Now the data in the table is cast to the common type int64 (7 columns) and object (5 columns).

#### Duplicate processing

Using the method duplicated (), let's see the number of duplicate rows in the table.

In [15]:
data_clients.duplicated().sum()

54

54 lines are completely duplicated. Let's remove them using the method drop_duplicates ().

In [16]:
data_clients = data_clients.drop_duplicates().reset_index(drop = True)

In [17]:
data_clients.duplicated().sum()
# to check that removal was success

0

Now let's find duplicates, case sensitive. We will only search for them in columns with the object data type.

In [18]:
data_clients['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

It can be seen that each type of education has 3 spellings. We will use the method str.lower (), this will bring all spellings to lowercase.

In [19]:
data_clients['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Words is written in lowercase in this column. No changes required.

In [20]:
data_clients['income_type'].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Words is written in lowercase in this column. No changes required.

In [21]:
data_clients['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

It can be seen many purposes for which the loan is taken. It will be necessary to highlight the main purposes and apply lemmatization.

We process duplicates in only one column 'education', case sensitive.

In [22]:
data_clients['education_lowercase'] = data_clients['education'].str.lower() 
data_clients['education_lowercase'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education_lowercase, dtype: int64

**Conclusion on this point:** 54 rows of the table were duplicates, these duplicates were deleted. Also the case sensitive duplicates were in the 'education' column. Added case-sensitive manual duplicate search. Brought everything to lowercase using the method str.lower (). Duplicates in the types of education could appear as a result of manual data entry or downloading data from different databases, and each of them, in turn, used its own spelling.

#### Lemmatization

In [23]:
m = Mystem()
def purpose_lemmas(purpose):
    lemma = ' '.join(m.lemmatize(purpose))
    return lemma
data_clients['purpose_lemmas'] = data_clients['purpose'].apply(purpose_lemmas)
# using the method apply() added the new column purpose_lemmas'. 
data_clients['purpose_lemmas'].value_counts()

автомобиль \n                                    972
свадьба \n                                       793
на   проведение   свадьба \n                     773
сыграть   свадьба \n                             769
операция   с   недвижимость \n                   675
покупка   коммерческий   недвижимость \n         662
операция   с   жилье \n                          652
покупка   жилье   для   сдача \n                 652
операция   с   коммерческий   недвижимость \n    650
жилье \n                                         646
покупка   жилье \n                               646
покупка   жилье   для   семья \n                 638
строительство   собственный   недвижимость \n    635
недвижимость \n                                  633
операция   со   свой   недвижимость \n           627
строительство   жилой   недвижимость \n          625
покупка   недвижимость \n                        621
покупка   свой   жилье \n                        620
строительство   недвижимость \n               

Conventionally, all purposes of the loan can be divided into 4 categories: wedding, real estate, car and education. The purpose of the "repair" loan can be attributed to real estate, since this operation is directly related to it. All operations with the purchase of commercial and non-commercial real estate, as well as construction, we will refer to the category of real estate, since the essence of these goals is the same.

In [24]:
# Let's write a loop that will go through this column and write the result in another column

def purpose_category(column):
    if 'ремонт' in column:
        return 'недвижимость'
    if 'жилье' in column:
        return 'недвижимость'
    if 'недвижимость' in column:
        return 'недвижимость'
    if 'автомобиль' in column:
        return 'автомобиль'
    if 'образование' in column:
        return 'образование'
    if 'свадьба' in column:
        return 'свадьба'
data_clients['purpose_category'] = data_clients['purpose_lemmas'].apply(purpose_category)
data_clients['purpose_category'].value_counts()
display(data_clients)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,purpose_lemmas,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее,покупка жилье \n,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее,приобретение автомобиль \n,автомобиль
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее,покупка жилье \n,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее,дополнительный образование \n,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее,сыграть свадьба \n,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,среднее,операция с жилье \n,недвижимость
21467,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,среднее,сделка с автомобиль \n,автомобиль
21468,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,среднее,недвижимость \n,недвижимость
21469,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,среднее,на покупка свой автомобиль \n,автомобиль


**Conclusion on this point:** in the process of lemmatization, 4 credit goals were identified. We created a "purpose_category" column, in which we wrote down the purpose of each loan in accordance with the selected categories.

#### Data categorization

When looking through all the columns of the table, it can be seen that education and marital status appear twice: family_status and family_status_id, education and education_id. This not only enlarges the table visually, but also makes it heavier, increases the processing time of the data in the table.

It is advisable to create "dictionaries" in which all variants of education / marital status and the IDs assigned to them will be collected.

In [25]:
education_dict = data_clients[['education_lowercase','education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict.sort_values('education_id')

,education_lowercase,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [26]:
family_status_dict = data_clients[['family_status','family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict.sort_values('family_status_id')

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Let's categorize the data by income level. We will divide the bank's clients into 4 groups using the method qcut () of splitting into quartiles.

In [55]:
pd.qcut(data_clients['total_income'], 4)

0        (195767.5, 2265604.0]
1         (107654.5, 142594.0]
2         (142594.0, 195767.5]
3        (195767.5, 2265604.0]
4         (142594.0, 195767.5]
                 ...          
21466    (195767.5, 2265604.0]
21467     (142594.0, 195767.5]
21468    (20666.999, 107654.5]
21469    (195767.5, 2265604.0]
21470    (20666.999, 107654.5]
Name: total_income, Length: 21471, dtype: category
Categories (4, interval[float64]): [(20666.999, 107654.5] < (107654.5, 142594.0] < (142594.0, 195767.5] < (195767.5, 2265604.0]]

**Conclusion on this point:** based on the intervals obtained, we define a low income - no more than 107654.5, an average income up to 142594, a high income level up to 195767.5, all that is more - a very high income level.

### We will answer the questions <a id="step3"></a>   

- Is there a correlation between the presence of children and the repayment of the loan on time?

In [27]:
# to answer this question, let's create a pivot table
child_pivod = data_clients.pivot_table(index=['children'], values='debt', aggfunc = ['count', 'mean'])
# name the new columns
child_pivod.set_axis(['количество заёмщиков', 'вероятность задолженности'], axis='columns', inplace=True)
# sort the pivot table by debt probability
child_pivod.sort_values(by = 'вероятность задолженности')

,количество заёмщиков,вероятность задолженности
children,,
5,9,0.000000
0,14107,0.075353
3,330,0.081818
1,4856,0.091639
2,2128,0.094925
4,41,0.097561


**Conclusion on this point:** we can neglect the statistics of borrowers who have 3, 4 and 5 children, since their percentage is negligible (less than 2% of the total number of borrowers). Accordingly, we will consider people who have no children, as well as borrowers who have 1 and 2 children.
- the borrower has no children - the probability of being debt is 7.5%
- the borrower has 1 child - the probability of being debt is 9.2%
- the borrower has 2 children - the probability of debt is 9.4%.
Based on this, we can conclude that having children negatively affects the return on credit. Moreover, with the increase in the number of children, the likelihood of debt formation on loans increases. The bank's childless clients are more reliable.

- Is there a correlation between marital status and loan repayment on time?

In [28]:
# to answer this question, let's create a pivot table
family_pivod = data_clients.pivot_table(index=['family_status'], values='debt', aggfunc = ['count', 'mean'])
# name the new columns
family_pivod.set_axis(['количество заёмщиков', 'вероятность задолженности'], axis='columns', inplace=True)
# sort the pivot table by debt probability
family_pivod.sort_values(by = 'вероятность задолженности')

,количество заёмщиков,вероятность задолженности
family_status,,
вдовец / вдова,959,0.065693
в разводе,1195,0.071130
женат / замужем,12344,0.075421
гражданский брак,4163,0.093202
Не женат / не замужем,2810,0.097509


**Conclusion on this point:** We cannot neglect any data. The smallest category "widow / widower" is more than 4% of the total number of borrowers.
Based on the data, we can conclude that the most numerous, and at the same time one of the most solvent categories, are married borrowers (the probability of debt is 7.5%). Also, good indicators of paying capacity were among widowers and divorced clients (6.6% and 7.1%, respectively). But people living in a civil marriage and single have the worst indicators (the probability of debt is 9.3% and 9.7%, respectively).

- Is there a correlation between income level and loan repayment on time?

In order to find out the correlation between income level and loan repayment on time, it is necessary to group borrowers by income level.

In [29]:
def income_category(x):
    if x < 107654.5:
        return 'low_inc'
    if 107654.5 < x < 142594.0:
        return 'middle_inc'
    if 142594.0 < x < 195767.5:
        return 'high_inc'
    else:
        return 'very_high_inc'
    
data_clients['income_category'] = data_clients['total_income'].apply(income_category)
data_clients['income_category'].value_counts()

very_high_inc    6445
low_inc          5368
high_inc         5245
middle_inc       4413
Name: income_category, dtype: int64

**Conclusion on this point:** From the data, it can be seen that the most numerous category are borrowers with an income level of more than 196 thousand, and the smallest with an average income level. Now let's check how the level of income affects the loan repayment on time.

In [30]:
# create a pivot table
income_pivod = data_clients.pivot_table(index=['income_category'], values='debt', aggfunc = ['count', 'mean'])
# name the new columns
income_pivod.set_axis(['количество заёмщиков', 'вероятность задолженности'], axis='columns', inplace=True)
# sort the pivot table by debt probability
income_pivod.sort_values(by = 'вероятность задолженности')

,количество заёмщиков,вероятность задолженности
income_category,,
very_high_inc,6445,0.074631
low_inc,5368,0.079545
high_inc,5245,0.085415
middle_inc,4413,0.087242


Unexpected conclusions: the most reliable were not only people with very high incomes, but also those with low ones. The probability of debt is 7.4% and 7.9%, respectively.
The most dangerous category for the bank turned out to be customers with low income. They have the highest probability of being indebted at 8.7%. People with a high level of income are also dangerous, their probability of indebtedness is not much lower than that of the previous group - 8.5%. Based on this, we can conclude that there is no direct correlation between the level of income and the return of loan funds.

- How do different purposes of a loan affect its return on time?

In [31]:
# create a pivot table
purpose_pivod = data_clients.pivot_table(index=['purpose_category'], values='debt', aggfunc = ['count', 'mean'])
# name the new columns
purpose_pivod.set_axis(['количество заёмщиков', 'вероятность задолженности'], axis='columns', inplace=True)
# sort the pivot table by debt probability
purpose_pivod.sort_values(by = 'вероятность задолженности')

,количество заёмщиков,вероятность задолженности
purpose_category,,
недвижимость,10814,0.072314
свадьба,2335,0.079657
образование,4014,0.092177
автомобиль,4308,0.093547


**Conclusion on this point:**
1. The best way to return loans taken on real estate. About 50% of people take out a loan for this purpose and return this money very well. It can be assumed that the population takes this type of lending more seriously. In this case, the real estate is pledged by the bank and in the event of a debt, the bank has the right to sell the borrower's apartment.
2. Weddings are a small but sure target. We can assume that in most cases, weddings "pay off" and the newlyweds have the opportunity to repay the existing loan ahead of schedule.
3. The situation is worse with loans for education. Perhaps the fact is that young people take loans for education.
4. The "worst" category is a car, the probability of debt is 9.3%. Perhaps this is due to the high level of road accidents in Russia. In common case, in such incidents, a borrower incurs additional expenses for car repairs. Also borrowers oblige themselves to additional expenses for a car's maintenance: fuel, parking, insurance.

### The general conclusion <a id="step4"></a>   

В первую очередь нужно указать кредитному отдела банка не некорректность данных относительно трудового стажа заёмщиков, "отрицательных детей" и семей с 20-ю детьми. Это позволит исключить эту проблему в будущем.

Исходя из проведенного анализа данных можно сделать вывод, что есть прямая зависимость между наличием детей и возвратом кредита в срок. Чем меньше детей у клиента, тем выше вероятность своевременного возврата кредитных средств. Данный вывод логичен, так как дети влекут за собой существенные расходы. Кредитному отделу банка стоит более детально проверять уровень дохода и кредитную историю семей с детьми.

Также наблюдается зависимость между семейным положением и возвратом кредита в срок. Клиенты из категории "не женат / не замужем" хуже всего возращают кредиты, а люди состоящие в браке наоборот. Вероятно, это связано с тем, что при потере работы одним из супругов, кредитные обязательства на себя берёт его партнер. Одинокие же клиенты не могут рассчитывать на такую помощь.

Зависимость между уровнем дохода и возвратом кредита в срок не выявлено. Это связано с тем, что сумма ежемесячного платежа расчитывается исходя из уровня дохода. Банк не выдаёт заемщикам "непосильные" кредиты. Соответсвенно размер кредита прямопропорционален уровню дохода.

Выявлено, что большинство заёмщиков берёт кредит на недвижимость. Вероятность задолженности по данной категории одна из самых минимальных. Большинство клиентов весьма серьёзно относятся к данному типу кредитования, так как в случае задолженности они могут остаться без жилья. Хуже всего возращают кредитные средства, взятые на образование и автомобиль. Кредитному отделу банка стоит обратить внимание на данные категории и более детально изучать заёмщиков, берущих кредит на эти цели.

Таким образом, самый надёжный заёмщик: "женат / замужем" без детей, цель кредита недвижимость.
А самый ненадёжный - "не женат / не замужем", имеющий 1-го ребёнка и более, цель кредита образование или покупка автомобиля.

First of all, it is necessary to indicate to the credit department of the bank that the data on the length of experience of borrowers, "negative children" and families with 20 children are not correct. This will eliminate this problem in the future.

Based on the analysis of the data, we can conclude that there is a direct correlation between the presence of children and the repayment of the loan on time. The fewer children a client has, the higher the likelihood of a timely return of loan funds. This conclusion is logical, since children entail significant costs. The credit department of the bank should check in more detail the level of income and credit history of families with children.

There is also the correlation between marital status and loan repayment on time. Clients from the "single / not married" category are the worst in loans returning, while married people are the opposite. This is probably due to the fact that when one of the spouses loses his job, his partner takes on the loan obligations. Lonely clients cannot count on such help.

The correlation between income and loan repayment on time has not been identified. This is due to the fact that the amount of the monthly payment is calculated based on the level of income. The bank does not issue "unbearable" loans to borrowers. Accordingly, the size of the loan is directly proportional to the level of income.

It was revealed that the majority of borrowers take out a loan for real estate. The probability of debt for this category is one of the lowest. Most clients are very serious about this type of lending, as in case of debt, they may be become homeless. The worst return on loans for education and cars. The bank's credit department should pay attention to these categories and consider in more detail the borrowers who take out a loan for these purposes.

Thus, the most reliable borrower: "married" without children, the purpose of the loan is real estate.
And the most unreliable - "not married / not married", having 1 child or more, the purpose of the loan is education or buying a car.